In [ ]:
import sys
sys.path.append("/storage/vbutoi/projects/")
sys.path.append("/storage/vbutoi/libraries/")
from ionpy.analysis import ResultsLoader
import pathlib
import seaborn as sns
import pandas as pd
sns.set_context('talk')

sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage',
))

# Results loader object does everything
rs = ResultsLoader()
root = "/storage/vbutoi/scratch/ESE"

In [ ]:
path_list = [
    # "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_SoftDiceLoss",
    # "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_PixelCELoss_wSVLS" # SVLS Models Seed 40
    # "/storage/vbutoi/scratch/ESE/calibration/07_23_24_ACDC_CalibrationSuite_TempScaling_HardGT", # Calibrated from above.
    # "/storage/vbutoi/scratch/ESE/calibration/07_23_24_ACDC_CalibrationSuite_IBTS_HardGT", # Calibrated from above.
    # "/storage/vbutoi/scratch/ESE/calibration/07_23_24_ACDC_CalibrationSuite_LTS_HardGT", # Calibrated from above.
    # "/storage/vbutoi/scratch/ESE/calibration/07_23_24_ACDC_CalibrationSuite_Popcorn_HardGT", # Calibrated from above.
]

df_list = []
for path in path_list:
    try:
        dfc = rs.load_configs(
            path,
            properties=False,
        )
        df = rs.load_metrics(dfc)
        df_list.append(df)
    except Exception as e:
        print(e)
df = pd.concat(df_list)

In [ ]:
for ikey in df.keys():
    print(ikey)

In [ ]:
# For num_examples, where it is NaN make it 20.
if 'num_examples' in df.keys():
    df['num_examples'] = df['num_examples'].fillna(20)
else:
    df['num_examples'] = 20

In [ ]:
# Get the path where num_examples is 20.
df[df['num_examples'] == 20]['path'].unique()

In [ ]:
# Sort the df by the seed.
df = df.sort_values(by='seed')
# Change the type of the seed column to a string
df['seed'] = df['seed'].astype(str)

In [ ]:
g = sns.relplot(
    data=df,
    x='epoch',
    y='loss',
    hue='seed',
    style='num_examples',
    col='phase',
    kind='line',
    height=12,
)

# Set column spacing
g.fig.subplots_adjust(wspace=0.05)
# g.set(ylim=(0.0, 0.25))

In [ ]:
# We want to make a custom palette that will differentiate across methods. Specifically, if the method
# is Popcorn_Scaling then we want it to be from the 'Blues' family otherwise we want it to be from the 'YlOrBr' family.
# custom_palette = {df['opt_method'].unique()[i]: sns.color_palette('Blues', n_colors=len(df['opt_method'].unique()))[i] if 'Popcorn_Scaling' in df['opt_method'].unique()[i] else sns.color_palette('YlOrBr', n_colors=len(df['opt_method'].unique()))[i] for i in range(len(df['opt_method'].unique()))}

In [ ]:
# g = sns.relplot(
#     data=df,
#     x='epoch',
#     y='dice_score',
#     hue='seed',
#     col='phase',
#     kind='line',
#     height=12,
#     # palette=custom_palette
# )

# # Set column spacing
# g.fig.subplots_adjust(wspace=0.05)
# # g.set(ylim=(0.0, 0.25))

In [ ]:
num_examples_df = df[df['num_examples'] == 1]

In [ ]:
g = sns.relplot(
    data=num_examples_df,
    x='epoch',
    y='ece_loss',
    hue='seed',
    style='num_examples',
    col='phase',
    kind='line',
    height=12,
    # palette=custom_palette
)

# Set column spacing
# Set the y axis to be between 0.008 and 0.001
g.set(ylim=(0.0005, 0.008))
g.fig.subplots_adjust(wspace=0.05)